In [1]:
import datetime
import sys
import os

import IPython
import IPython.display
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from sklearn.preprocessing import MinMaxScaler

import src.preprocessing_3days
from src.preprocessing_3days import series_to_supervised, preprocess
from src.functions import load_data, TimeSeriesTensor, create_evaluation_df, plot_train_history, validation, save_model, load_model

np.set_printoptions(threshold=sys.maxsize)
pd.options.display.max_seq_items = 2000

In [2]:
def train_test_split(df, n_test):
    if len(df) < 8760:
        n_test = round(len(df) * 0.2)
    test_df = df.copy()[-(n_test+71):]
    train_df = df.copy()[:-(len(test_df)-71)]
    return train_df, test_df

In [3]:
def MIMO_fulldata_preparation(df, n_test=4380, T=72, HORIZON=72):
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    df = preprocess(df, 'Belgium')
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    #print(train_df.columns)
    #tensor_structure = {'X':(range(-T+1, 1), train_df.columns[:1]), 'X2':(range(1, 73), train_df.columns[1:6]), 'static':(None, train_df.columns[6:])}
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns[:1]), 'X2':(range(1, 73), train_df.columns[1:])}
    #tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    #print(tensor_structure[0])
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler

In [57]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]

In [64]:
metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dX_test = []
dX_scaler = []
for i in pbar(range(1,28)):
    filename = '../data/Columbia_clean/Residential_'+str(i)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df)
    baseline = Baseline(label_index = 0)
    baseline.compile(loss=tf.losses.MeanSquaredError(), metrics=[tf.metrics.MeanSquaredError()])
    predictions_B = baseline.predict(test_inputs['X'])
    eval_df_B = create_evaluation_df(predictions_B.reshape(-1,24), test_inputs, 24, y_scaler)
    mape = validation(eval_df_B['prediction'], eval_df_B['actual'], 'MAPE')
    mae = validation(eval_df_B['prediction'], eval_df_B['actual'], 'MAE')
    rmse = validation(eval_df_B['prediction'], eval_df_B['actual'], 'RMSE')
    metrics.loc[i-1] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/global/baseline.csv')

100% |########################################################################|


In [4]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 72


FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(HORIZON, 14)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])




metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dT_train = []
dX_test = []
dX_scaler = []
for i in pbar(range(1,29)):
    filename = '../data/Columbia_clean/Residential_'+str(i)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df, n_test=4380, T=72, HORIZON=72)
    dX_train.append(tf.concat([train_inputs['X'],train_inputs['X2']], axis=2))
    dT_train.append(train_inputs['target'])
    dX_test.append(test_inputs)
    dX_scaler.append(y_scaler)
global_inputs_X = tf.concat(dX_train, 0)
global_inputs_T = tf.concat(dT_train, 0)
#test_inputs = pd.concat(dn_test, axis=1)

# full data LSTM MIMO compilation and fit
FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
        
history = FULL_LSTMIMO.fit(global_inputs_X, global_inputs_T, batch_size=1500, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)
save_model(FULL_LSTMIMO, 'Columbia_model')

for i in range(0,28):
    concat_input = tf.concat([dX_test[i]['X'],dX_test[i]['X2']], axis=2)
    FD_predictions = FULL_LSTMIMO.predict(concat_input)
    FD_eval_df = create_evaluation_df(FD_predictions, dX_test[i], HORIZON, dX_scaler[i])
    mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
    mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
    rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
    metrics.loc[i] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/Columbia/global/3days/revised2_LSTM.csv')

100% |########################################################################|


Train on 420321 samples, validate on 74175 samples
Epoch 1/100
328500/420321 [======================>.......] - ETA: 1:32 - loss: 0.0129 - mean_squared_error: 0.0129

KeyboardInterrupt: 

In [14]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 72

#FULL_LSTMIMO = tf.keras.models.Sequential([
#    # Shape [batch, time, features] => [batch, time, lstm_units]
#    tf.keras.layers.LSTM(32, input_shape=(24, 15)),
#    # Shape => [batch, time, features]
#    tf.keras.layers.Dense(HORIZON)
#])

FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 15), return_sequences=True),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.LSTM(32),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])


metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dT_train = []
dX_test = []
dX_scaler = []
for i in pbar(range(1,28)):
    filename = '../data/Columbia_clean/Residential_'+str(i)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df)
    dX_train.append(train_inputs['X'])
    dT_train.append(train_inputs['target'])
    dX_test.append(test_inputs)
    dX_scaler.append(y_scaler)
global_inputs_X = tf.concat(dX_train, 0)
global_inputs_T = tf.concat(dT_train, 0)
#test_inputs = pd.concat(dn_test, axis=1)

# full data LSTM MIMO compilation and fit
FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
        
history = FULL_LSTMIMO.fit(global_inputs_X, global_inputs_T, batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)
for i in range(1,28):
    FD_predictions = FULL_LSTMIMO.predict(dX_test[i-1]['X'])
    FD_eval_df = create_evaluation_df(FD_predictions, dX_test[i-1], HORIZON, dX_scaler[i-1])
    mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
    mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
    rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
    metrics.loc[i-1] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/global/3days/stacked_LSTM.csv')

100% |########################################################################|


Epoch 1/100


ValueError: in user code:

    /home/ubuntu/anaconda3/envs/evgeny/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/ubuntu/anaconda3/envs/evgeny/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ubuntu/anaconda3/envs/evgeny/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ubuntu/anaconda3/envs/evgeny/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ubuntu/anaconda3/envs/evgeny/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /hom

In [15]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

#ConditionalRNN = tf.keras.models.Sequential([
#    # Shape [batch, time, features] => [batch, time, lstm_units]
#    tf.keras.layers.LSTM(32, input_shape=(24, 15)),
#    # Shape => [batch, time, features]
#    tf.keras.layers.Dense(HORIZON)
#])

ConditionalRNN = Sequential(layers=[ConditionalRNN(32, cell='LSTM'),
                                    Dense(HORIZON)])


metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dc1_train = []
dc2_train = []
dT_train = []
dX_test = []
dc1_test = []
dc2_test = []
dX_scaler = []
for i in pbar(range(1,28)):
    filename = '../data/Columbia_clean/Residential_'+str(i)+'.csv'
    df = pd.read_csv(filename, index_col=0)
    train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler = ConditionalRNN_data_preparation(df)
    dX_train.append(train_inputs['X'])
    dT_train.append(train_inputs['target'])
    dc1_train.append(train_c1)
    dc2_train.append(train_c2)
    dc1_test.append(test_c1)
    dc2_test.append(test_c2)
    dX_test.append(test_inputs)
    dX_scaler.append(y_scaler)
global_inputs_X = tf.concat(dX_train, 0)
global_inputs_c1 = tf.concat(dc1_train, 0)
global_inputs_c2 = tf.concat(dc2_train, 0)
global_inputs_T = tf.concat(dT_train, 0)
#test_inputs = pd.concat(dn_test, axis=1)

# full data LSTM MIMO compilation and fit
ConditionalRNN.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
        
history = ConditionalRNN.fit([global_inputs_X, global_inputs_c1, global_inputs_c2], global_inputs_T, batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)
for i in range(1,28):
    FD_predictions = ConditionalRNN.predict([dX_test[i-1]['X'],dc1_test[i-1],dc2_test[i-1]])
    FD_eval_df = create_evaluation_df(FD_predictions, dX_test[i-1], HORIZON, dX_scaler[i-1])
    mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
    mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
    rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
    metrics.loc[i-1] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/Columbia/global/conditional_LSTM.csv')

100% |########################################################################|


Epoch 1/100
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by pass

In [4]:
def ConditionalRNN_data_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Canada')
    c2 = series_to_supervised(df)
    c1 = df.iloc[:,1:9]
    c1 = c1.loc[c2.index]
    short_df = df.iloc[:,[0,1,-4,-3,-2,-1]].copy()
    #check how to formulate more correctly
    short_df = short_df[146:]
    train_df, test_df = train_test_split(short_df, n_test)
    train_c1, test_c1 = train_test_split(c1, n_test)
    train_c2, test_c2 = train_test_split(c2, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    train_c2[['value(t-24)']] = y_scaler.fit_transform(train_c2[['value(t-24)']])
    train_c2[['value(t-168)']] = y_scaler.fit_transform(train_c2[['value(t-168)']])
    X_scaler = MinMaxScaler()
    train_df[train_df.columns] = X_scaler.fit_transform(train_df)
    test_df[train_df.columns] = X_scaler.fit_transform(test_df)
    tensor_structure = {'X':(range(-T+1, 1), ['value','fractional hour_sin','fractional hour_cos','day of year_sin','day of year_cos'])}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    train_c1 = c1.reindex(train_inputs.dataframe.index)
    train_c2 = c2.reindex(train_inputs.dataframe.index)
    test_c1 = c1.reindex(test_inputs.dataframe.index)
    test_c2 = c2.reindex(test_inputs.dataframe.index)
    return train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler